# Parkinson detection with a shallow neural network

In this notebook I will cover the implementation for solving the parkinson detection problem from spiral drawings with a neural network. Note that this notebook contains twoo of the three propoused solutions, so in order to get a better understanding of the problem check the other two as well as the documentation abailable that goes more in depth.

In [10]:
import pandas as pd
import numpy as np
import glob
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from IPython.display import display
from IPython.display import Image
#import featureExtraction as fe
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics


# Feature Selection

When tackling an unbalanced data set it's important to do a proper feature selection process. First it's important to understand that the data is a time series and we will have to manage this appropriately. Since we are working with drawings and the main part parkinson will affect this drawings is in the inconsistency of the lines, I decided to measure the variance and the absolute difference from the highest and lowest values in a subset of data.

Let's take a look  at the values our dataset has to offer as seen in the readme.txt file:

----------------
X ; Y; Z; Pressure; GripAngle; Timestamp; Test ID


Test ID: 
0: Static Spiral Test ( Draw on the given spiral pattern)
1: Dynamic Spiral Test ( Spiral pattern will blink in a certain time, so subjects need to continue on their draw)
2: Circular Motion Test (Subjectd draw circles around the red point)

----------------

Let's start by deffining a function that loads the files from the folders where it is stored.

## Accessing the folders
As can be seen using glob and and pandas we can make a list that stores the information of all the files in the directory.

In [11]:
control_path = "Data/control"
parkinson_path = "Data/parkinson"

def getFiles(path):
    all_files = glob.glob(path + "/*.txt")
    data = []

    for file in all_files:
        df = pd.read_csv(file,sep=";")
        data.append(df)
    
    return data

## Splitting data by test_id
The next step is for each file to separate the data by test, so we don't mix up our data. We will iterate line from lines making a list until we find that the test ID is different if we to we save the data we got up until this moment and start a new list.

In [12]:
def splitByTest(df):
    data = []
    test = df.values[0][6]
    split = []
    cont = 0
    for i in range(df.shape[0]):
        if df.values[i][6] == test:
            split.append(df.values[i])
        else:
            aux = pd.DataFrame(split)
            data.append(aux)
            split = []
            test = df.values[i][6]
            split.append(df.values[i])
    data.append(split)
    return pd.DataFrame(data)

## Extracting the desired features
The next step is to extract the features we want from our data set, since we are only interested in the variance and difference of the first 5 columns we will extract it using numpy we will also add to which test the data pertains to.

In [13]:
def extractFeatures(df):
    features = []
    
    features.append(np.var( df.values[:,0]))
    features.append(np.var( df.values[:,1]))
    features.append(np.var( df.values[:,2]))
    features.append(np.var( df.values[:,3]))
    features.append(np.var( df.values[:,4]))
    features.append(np.argmax( df.values[:,0]) - np.argmin( df.values[:,0]))
    features.append(np.argmax( df.values[:,1]) - np.argmin( df.values[:,1]))
    features.append(np.argmax( df.values[:,2]) - np.argmin( df.values[:,2]))
    features.append(np.argmax( df.values[:,3]) - np.argmin( df.values[:,3]))
    features.append(np.argmax( df.values[:,4]) - np.argmin( df.values[:,4]))
    features.append(df.values[0][6])
    return features

## Putting it all together

Now we will put it all together, and in order to maximize our learning potential we will split each test data in different subsets so we can get more information. One other thing we will do, is make different sizes of subset from the control data and the parkinson data. It's important not to skew it too much, keep it below 2:1 ratio in the number of splits. The less we alter the data the better.

In [14]:
def getControlFeatures():
    control_data = []
    control = getFiles(control_path)
    for i in control:
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,700)
            for j in last_df:
                control_data.append(extractFeatures(j))
    return pd.DataFrame(control_data)

def getParkinsonFeatures():
    parkinson_data = []
    parkinson = getFiles(parkinson_path)
    for i in parkinson:       
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,400)
            for j in last_df:
                parkinson_data.append(extractFeatures(j))
    return pd.DataFrame(parkinson_data)

## Getting the classes

Last but not least we will get the classes, this is easy we just make an array of 0s and 1s based on how many control and parkinson clases we have.

In [15]:
def getClasses(control, parkinson):
    classes = []
    for i in range(control.shape[0]):
        classes.append(0)
    for i in range(parkinson.shape[0]):
        classes.append(1)
    res = np.array(classes)

    return res

# Neural Network

We have set the functions to get the features in place, now is time to create our multy layer perceptron. We will set it up so it can recive as many hidden layers as we want. This will also allow you to do you own testing

In [16]:
def MLP(data, classes, test_data, test_classes,input_size, layer_sizes):
    tf.reset_default_graph()
    
    #creating The place holders for our input and output
    x = tf.placeholder(tf.float32, [None, input_size])
    y = tf.placeholder(tf.float32, [None, 2])
    #the lists of weights and biases will start empty but the layers will begin with x
    weights = []
    biases = []
    layers = [x]
    
    #set a variable to stor the size of the last layer
    last_layer = input_size
    
    #Iterate through the layers so they all feed into eachother
    for layer, neurons in enumerate(layer_sizes):  
        weights = weights + [tf.Variable(tf.zeros([last_layer, neurons]),name="layer" + str(len(layers)) + "_weights")]
                       
        biases = biases + [tf.Variable(tf.zeros([neurons]),name="layer" + str(len(layers)) + "_biases")]
                       
        layers += [tf.sigmoid(tf.add(tf.matmul(layers[len(layers)-1],weights[len(weights)-1]),biases[len(biases)-1]))]
        last_layer = neurons
    
    #Add the the weights biases and last sigmoid fucntion to get the prediction
    weights = weights + [tf.Variable(tf.zeros([last_layer, 2]),name="last_layer_weights")]
    biases = biases +[tf.Variable(tf.zeros([2]),name="last_layer_biases")]                                                 
    prediction = tf.add(tf.matmul(layers[len(layers)-1],weights[len(weights)-1]),biases[len(biases)-1])
    
    #Both of this loss functions work properly although the sigmoid one gives slightly better results
    #loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction)
    
    #the predicted class will be the one with the highest value
    class_pred = tf.argmax(prediction, axis=1)
    
    #set upt the optimizer
    lr = 0.0005
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
    #Initialize all the variables
    init = tf.global_variables_initializer()

    training_epochs = 1000 *14
    train_n_samples = data.shape[0]
    display_step = 2000

    mini_batch_size = 250
    n_batch = train_n_samples // mini_batch_size + (train_n_samples % mini_batch_size != 0)
    
    #Start training
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            i_batch = (epoch % n_batch)*mini_batch_size
            batch = data[i_batch:i_batch+mini_batch_size, :], classes[i_batch:i_batch+mini_batch_size, :]
            sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
            if (epoch+1) % display_step == 0:
                cost = sess.run(loss, feed_dict={x: batch[0], y: batch[1]})
                acc = np.sum(np.argmax(test_classes, axis=1) == sess.run(class_pred, feed_dict={x: test_data}))/test_classes.shape[0]
                print("Epoch:", str(epoch+1), "Error:", np.mean(cost), "Accuracy:", acc)
        parameters = sess.run(weights+biases)
        test_predictions = sess.run(class_pred, feed_dict={x: test_data})
    return parameters, test_predictions, acc

## Setting up the data

Now let's see how we set up our data in order the feed it to the MLP. First lets load the data.

In [17]:
control = getControlFeatures()
park = getParkinsonFeatures()
target = getClasses(control,park)

Now we have to put it together in order to get our train and test split. We will use numpy to concatenate them.

In [18]:
all_data = np.concatenate((control,park),axis= 0)

We can get our split in two ways, one is using the StratifiedKFold function, and the other the train_test_split function. I will put the code to both below but we will only use the train_test_split since result were significantly better. It is important to use the random state sin when concatenating the data we are following a very strict order.

In [19]:
'''
skf = StratifiedKFold(n_splits=50,shuffle=True)

for train_index, test_index in skf.split(all_data, target):
    train_data, test_data = all_data[train_index], all_data[test_index]
    train_target, test_target = target[train_index], target[test_index]
'''
train_data, test_data, train_target, test_target = train_test_split(all_data,target,
                                                                    test_size= 0.3, random_state=30 )

Now, since our last layer has 2 nodes in it we have to modify our target information to also have 2 columns. The best way of doing this is using the one hot techniche.

In [20]:
n_classes = np.unique(train_target).shape[0]
train_target = np.eye(n_classes)[train_target]
test_target = np.eye(n_classes)[test_target]

Final touches before feeding the data to the MLP. We will get the input size by checking how many columns our training data has, and we will also define the amount and the size of the hidden layers, for this particular case, one hidden layer of size seven works best.

In [21]:
input_size = np.shape(train_data)[1]
print(input_size)
layers = np.array([7])

11


## Testing the MLP
Now we can feed all this to our MLP and check the results.

In [22]:
params, test_preds, _  = MLP(train_data, train_target, test_data, test_target,input_size, layers)
cont1 =np.count_nonzero(test_preds)
cont0 = len(test_preds) - cont1
print(confusion_matrix(test_preds, np.argmax(test_target, axis=1)))

Epoch: 2000 Error: 0.5426574 Accuracy: 0.7096481682988756
Epoch: 4000 Error: 0.4662113 Accuracy: 0.7214726151614074
Epoch: 6000 Error: 0.43344012 Accuracy: 0.7977511788175553
Epoch: 8000 Error: 0.36063316 Accuracy: 0.8176278563656147
Epoch: 10000 Error: 0.3869553 Accuracy: 0.8121508886470802
Epoch: 12000 Error: 0.38533595 Accuracy: 0.830177729416032
Epoch: 14000 Error: 0.33761773 Accuracy: 0.8355458832063838
[[ 5545  2230]
 [ 2304 17491]]


As can be seen above with barely any training time we can solve this problem with pretty good results

# Neural Network with a varying learning rate

One of the biggest problems with the solution proposed above is the fact that all the data is processed as one and thus when making splits of different sizes we are also making testing sets with tempered qualities. In order to tackle this problem I have come up with this solution.

## Changes in the Features

The first change is that we will reduce the amount of splits by 10. Although for the training dataset we will continue the 7:4 ratio our testing data will be gotten from a  different set of files and both will be split in the exact same size, 40. wich in order not to favour the control data I have chosen the size of the parkinson data split with the hope to highlight the value of this solution

In [23]:
control_path = "DataSplit/control"
parkinson_path = "DataSplit/parkinson"
control_pathT = "DataSplit/a"
parkinson_pathT = "DataSplit/b"

Let's redefine our function to extract the features from the data.

In [24]:
def getControlFeatures():
    control_data = []
    control = getFiles(control_path)
    for i in control:
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,70)
            for j in last_df:
                control_data.append(extractFeatures(j))
    return pd.DataFrame(control_data)

def getControlFeaturesTest():
    control_dataT = []
    control = getFiles(control_pathT)
    for i in control:
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,40)
            for j in last_df:
                control_dataT.append(extractFeatures(j))
    return pd.DataFrame(control_dataT)

def getParkinsonFeatures():
    parkinson_data = []
    parkinson = getFiles(parkinson_path)
    for i in parkinson:
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,40)
            for j in last_df:
                parkinson_data.append(extractFeatures(j))
    return pd.DataFrame(parkinson_data)

def getParkinsonFeaturesTest():
    parkinson_dataT = []
    parkinson = getFiles(parkinson_pathT)
    for i in parkinson:
        new_df = splitByTest(i)
        for k in range(new_df.shape[0]):
            last_df = np.array_split(i,40)
            for j in last_df:
                parkinson_dataT.append(extractFeatures(j))
    return pd.DataFrame(parkinson_dataT)

## Changes to the MLP

As you will be able to see the changes are not too crazy will add an if in our training cycle that every X amount of epoch it will reduce the learning rate by half. This modification will allow the optimizers to take bigger steps to escape the local minima of predicting everything as 1 and after it has escaped by reducing the learning rate it will start to learn the details it needs in order to get the control cases too.

In [25]:
def custom_MLP(data, classes, test_data, test_classes,input_size, layer_sizes):
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, input_size])
    y = tf.placeholder(tf.float32, [None, 2])
    weights = []
    biases = []
    layers = [x]
    
    last_layer = input_size
    for layer, neurons in enumerate(layer_sizes):  
        weights = weights + [tf.Variable(tf.zeros([last_layer, neurons]),name="layer" + str(len(layers)) + "_weights")]
                       
        biases = biases + [tf.Variable(tf.zeros([neurons]),name="layer" + str(len(layers)) + "_biases")]
                       
        layers += [tf.sigmoid(tf.add(tf.matmul(layers[len(layers)-1],weights[len(weights)-1]),biases[len(biases)-1]))]
        last_layer = neurons
        
    weights = weights + [tf.Variable(tf.zeros([last_layer, 2]),name="last_layer_weights")]
    biases = biases +[tf.Variable(tf.zeros([2]),name="last_layer_biases")]                                                 
    prediction = tf.add(tf.matmul(layers[len(layers)-1],weights[len(weights)-1]),biases[len(biases)-1])

    #loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction)
    class_pred = tf.argmax(prediction, axis=1)
    
    #starting LR
    lr = 0.005
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

    init = tf.global_variables_initializer()
    
    #The number of epochs has also been upped by a bit since this is a more complex problem
    training_epochs = 1000 *62
    train_n_samples = data.shape[0]
    display_step = 2000

    mini_batch_size = 100
    n_batch = train_n_samples // mini_batch_size + (train_n_samples % mini_batch_size != 0)
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            i_batch = (epoch % n_batch)*mini_batch_size
            batch = data[i_batch:i_batch+mini_batch_size, :], classes[i_batch:i_batch+mini_batch_size, :]
            sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
            if (epoch+1) % display_step == 0:
                cost = sess.run(loss, feed_dict={x: batch[0], y: batch[1]})
                acc = np.sum(np.argmax(test_classes, axis=1) == sess.run(class_pred, feed_dict={x: test_data}))/test_classes.shape[0]
                print("Epoch:", str(epoch+1), "Error:", np.mean(cost), "Accuracy:", acc)
            #This if will reduces the learning rate in half every 150k epochs
            if epoch % 30000 == 0:
                lr = lr *0.5
                optimizer.learning_rate = lr
        parameters = sess.run(weights+biases)
        test_predictions = sess.run(class_pred, feed_dict={x: test_data})
    return parameters, test_predictions, acc

## loading the data
Since we have processed the training and the testing data in a better way we no longer need the train test split function and we can load them directly, The files that where chosen and testing data where chosen as random as to not to tamper with the dataset.

In [26]:
data = getControlFeatures()
park = getParkinsonFeatures()

test_control = getControlFeaturesTest()
test_park = getParkinsonFeaturesTest()

all_data = np.concatenate((data,park),axis= 0)
all_dataTest = np.concatenate((test_control,test_park),axis= 0)

target = getClasses(data,park)
targetTest = getClasses(test_control,test_park)

#One hot encoding
n_classes = np.unique(target).shape[0]
train_target = np.eye(n_classes)[target]
test_target = np.eye(n_classes)[targetTest]

## Feeding the data into the new MLP

Now we set up the last deatils and test how our new MLP performs.

In [27]:
input_size = np.shape(all_data)[1]
print(input_size)
layers = np.array([7])
params, test_preds, _  = custom_MLP(all_data, train_target, all_dataTest, test_target,input_size, layers)

print(confusion_matrix(test_preds, np.argmax(test_target, axis=1)))

11
Epoch: 2000 Error: 0.15407906 Accuracy: 0.7727272727272727
Epoch: 4000 Error: 0.29726428 Accuracy: 0.7727272727272727
Epoch: 6000 Error: 0.47202072 Accuracy: 0.7659090909090909
Epoch: 8000 Error: 1.0311526 Accuracy: 0.790340909090909
Epoch: 10000 Error: 0.16155408 Accuracy: 0.7659090909090909
Epoch: 12000 Error: 0.19617914 Accuracy: 0.7670454545454546
Epoch: 14000 Error: 0.37720585 Accuracy: 0.7693181818181818
Epoch: 16000 Error: 0.9635094 Accuracy: 0.7886363636363637
Epoch: 18000 Error: 0.16256982 Accuracy: 0.7852272727272728
Epoch: 20000 Error: 0.15093979 Accuracy: 0.7670454545454546
Epoch: 22000 Error: 0.30798447 Accuracy: 0.7465909090909091
Epoch: 24000 Error: 0.38509345 Accuracy: 0.7653409090909091
Epoch: 26000 Error: 1.0645844 Accuracy: 0.772159090909091
Epoch: 28000 Error: 0.1516212 Accuracy: 0.7551136363636364
Epoch: 30000 Error: 0.16407265 Accuracy: 0.7926136363636364
Epoch: 32000 Error: 0.37569228 Accuracy: 0.8022727272727272
Epoch: 34000 Error: 0.9489357 Accuracy: 0.79545

As we can observe even if the accuracy is not as good, it is barely 3% worse and we have managed to get a better regularization by not making different splits in the testing date for each class. I would consider these a far better solution. If we also look at the confusion matrix we see that it has learned perfectly well the difference from both classes.